In [8]:
from random import choice, choices
import base64
from Crypto.Cipher import AES

BLOCKSIZE = 16

def XOR(A: bytes, B: bytes):
    assert type(A) is bytes and type(B) is bytes and len(A) == len(B)
    return bytes([ a^b for a, b in zip(A, B) ])

def IntToLittleEndian(x: int, nbytes: int):
    assert type(x) is int and x >= 0
    out = bytes()
    while x >  0:
        out += bytes([ x & 0xFF ])
        x >>= 8
    
    assert len(out) <= nbytes
    out += bytes( [0] * ( nbytes-len(out) ) )
    
    return out

def LittleEndianToInt(x: bytes):
    assert type(x) is bytes
    out = 0
    byte_counter = 0
    while len(x) > 0:
        out += x[0] << byte_counter*8
        byte_counter += 1
        x = x[1:]
        
    return out

class CTR(object):
    def __init__(self, nonce: bytes, key: bytes):
        assert type(nonce) is bytes and len(nonce) < BLOCKSIZE
        self.nonce = nonce
        self.aes = AES.new(key, AES.MODE_ECB)
        
    def EncryptBlock(self, block: bytes, counter: int):
        assert type(counter) is int and counter >= 0
        assert type(block) is bytes and len(block) == BLOCKSIZE
        little_endian: bytes = IntToLittleEndian(counter, nbytes = BLOCKSIZE-len(self.nonce))
        key_block = self.aes.encrypt( self.nonce + little_endian )
        assert type(key_block) is bytes and len(key_block) == BLOCKSIZE
        return XOR(key_block, block)
    
    def DecryptBlock(self, block, counter: int):
        return self.EncryptBlock(block, counter)
    
    def EncryptBytes(self, plaintext: bytes, counter: int):
        assert type(plaintext) is bytes and len(plaintext) <= BLOCKSIZE
        assert type(counter) is int and counter >= 0
        little_endian: bytes = IntToLittleEndian(counter, nbytes = BLOCKSIZE-len(self.nonce))
        key_block = self.aes.encrypt( self.nonce + little_endian )
        assert type(key_block) is bytes and len(key_block) == BLOCKSIZE
        return XOR(key_block[:len(plaintext)], plaintext)
    
    def DecryptBytes(self, ciphertext: bytes, counter: int):
        return self.EncryptBytes(ciphertext, counter)
    
    def EncryptStream(self, plaintext: bytes, counter: int = 0):
        assert type(plaintext) is bytes
        stream = bytes(plaintext)
        assert len(stream) % BLOCKSIZE == 0
        ciphertexts = list()
        
        while len(stream) > 0:
            block =  stream[:BLOCKSIZE]
            stream = stream[BLOCKSIZE:]
            assert len(block) == BLOCKSIZE
            ciphertexts += [ self.EncryptBytes(block, counter) ]
            counter += 1
        
        ciphertext = b''.join(ciphertexts)
        assert len(ciphertext) == len(plaintext)
        return ciphertext
    
    def DecryptStream(self, plaintext: bytes, counter: int = 0):
        return self.EncryptStream(plaintext, counter)
    

In [72]:
class EncryptedHardDisk(object):
    nonce = b'A'*8
    
    def __init__(self, key: bytes, data: bytes, encrypt: bool = False):
        assert type(data) is bytes and len(data) % BLOCKSIZE == 0
        # Keep the CTR object and key because it will be require for the re-crypt function
        self.ctr_cipher = CTR(nonce = self.nonce, key = key)
        self.encrypted_data = self.ctr_cipher.EncryptStream(data)

    def DecryptAll(self, key):
        # Decrypt the whole disk and return
        # Require the key to be provided, even though the hard drive has the key,
        # because I think that's most realistic.
        return CTR(nonce = self.nonce, key = key).DecryptStream(self.encrypted_data)
    
    def EditBlock(self, new_plaintext: bytes, start: int):
        # start is the index of the block to recrypt
        assert start % BLOCKSIZE == 0 # make sure this is the beginning of a block
        assert len(new_plaintext) == BLOCKSIZE
        new_ciphertext = self.ctr_cipher.EncryptBlock(new_plaintext, counter = start//BLOCKSIZE)
        self.encrypted_data = self.encrypted_data[:start] + new_ciphertext + self.encrypted_data[start+BLOCKSIZE:]
        return new_ciphertext
        
        

In [73]:
import base64

original_key = b'YELLOW SUBMARINE'
with open('data.txt', mode = 'rb') as f:
    ciphertext = base64.decodebytes(f.read())
    plaintext = AES.new(original_key, AES.MODE_ECB).decrypt(ciphertext)
    pad = len(plaintext) % BLOCKSIZE
    plaintext += bytes([pad] * pad)
    disk = EncryptedHardDisk(key = original_key, data = plaintext)


In [74]:
# For each block, edit it to be all 0s
# The new ciphertext will be the mask that was XORed with the original data
# So just XOR that with the original ciphertext, to get the original plaintext

old_ciphertext = disk.encrypted_data
old_plaintext = list()

for n in range(len(old_ciphertext) // BLOCKSIZE):
    start, end = n*BLOCKSIZE, (n+1)*BLOCKSIZE
    block = old_ciphertext[start:end]
    bitmask = disk.EditBlock(bytes([0]*BLOCKSIZE), start)
    old_plaintext.append( XOR(bitmask, block) )
    
print(b''.join(old_plaintext))

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 